In [10]:
# import pandas as pd
# import psycopg
# from contextlib import contextmanager


# @contextmanager
# def get_connection(config):
#     """
#     Context manager to handle PostgreSQL connection lifecycle.
#     Ensures that the connection is closed after use.
#     """
#     conn = None
#     try:
#         conn = psycopg.connect(
#             dbname=config["dbname"],
#             user=config["user"],
#             password=config["password"],
#             host=config["host"],
#             port=config["port"],
#         )
#         yield conn
#         print(f"Connected to database {config['dbname']} successfully.")
#     except psycopg.OperationalError as e:
#         print(f"Error connecting to database {config['dbname']}: {e}")
#         raise
#     finally:
#         if conn:
#             conn.close()
#             print(f"Connection to database {config['dbname']} closed.")
#         else:
#             print(f"Failed to connect to database {config['dbname']}.")


# def fetch_column_names_for_query(conn, query):
#     """
#     Fetch column names based on the query, supporting complex queries like CTEs, joins, and window functions.
    
#     :param conn: A psycopg connection object.
#     :param query: SQL query to analyze.
#     :return: A list of column names.
#     """
#     try:
#         # Execute the query with LIMIT 0 to avoid fetching rows and only get column names
#         with conn.cursor() as cursor:
#             cursor.execute(f"{query} LIMIT 0;")
#             return [desc[0] for desc in cursor.description]
#     except Exception as e:
#         print(f"Error executing query to fetch column names: {e}")
#         raise


# def fetch_data_in_chunks(conn, query, chunk_size=10000, params=None):
#     """
#     Fetch data in chunks to handle large datasets.

#     :param conn: A psycopg connection object.
#     :param query: SQL query to execute.
#     :param chunk_size: Number of rows to fetch per chunk.
#     :param params: Optional parameters for the SQL query.
#     :return: A generator that yields chunks of data.
#     """
#     with conn.cursor(name="data_cursor") as cursor:
#         cursor.execute(query, params)
#         while True:
#             data = cursor.fetchmany(chunk_size)
#             if not data:
#                 break
#             yield data


# def run_query_with_dynamic_columns(db_configs, user_query, chunk_size=10000, params=None):
#     """
#     Execute a user query across multiple databases and return the results as a pandas DataFrame.

#     :param db_configs: List of dictionaries containing database connection parameters.
#     :param user_query: SQL query provided by the user.
#     :param chunk_size: Number of rows to fetch per chunk.
#     :param params: Optional parameters for the SQL query.
#     :return: A pandas DataFrame with the combined results from all databases.
#              The DataFrame includes a column for the database name.
#     """
#     dfs = []

#     for config in db_configs:
#         try:
#             with get_connection(config) as conn:
#                 # Fetch the correct column names based on the user query
#                 column_names = fetch_column_names_for_query(conn, user_query)

#                 # Fetch data in chunks and append to DataFrames list
#                 for chunk in fetch_data_in_chunks(conn, user_query, chunk_size, params):
#                     if chunk:
#                         # Validate that the number of columns matches the expected number
#                         if len(chunk[0]) != len(column_names):
#                             print(f"Warning: Number of columns in result from {config['dbname']} does not match the provided column names.")
#                             continue

#                         for row in chunk:
#                             if len(row) != len(column_names):
#                                 print(f"Warning: Row length mismatch in {config['dbname']}.")
#                                 continue

#                         # Create DataFrame and append to the list
#                         df = pd.DataFrame(chunk, columns=column_names)
#                         dfs.append(df)
#                     else:
#                         print(f"No data returned from {config['dbname']}.")
#         except Exception as e:
#             print(f"Error processing database {config['dbname']}: {e}")

#     # Concatenate all DataFrames into a single DataFrame
#     combined_df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

#     # Log message if no data was retrieved
#     if combined_df.empty:
#         print("No data was retrieved from the databases.")
    
#     return combined_df


# # Example usage
# db_configs = [
#     {
#         "dbname": "Employees",
#         "user": "postgres",
#         "password": "root",
#         "host": "localhost",
#         "port": "5432"
#     }
# ]

# # Sample advanced query (with CTE and window function)
# user_query = """
# WITH department_summary AS (
#     SELECT department, COUNT(emp_no) AS num_employees
#     FROM employees
#     GROUP BY department
# )
# SELECT e.emp_no, e.name, d.department, d.num_employees,
#        ROW_NUMBER() OVER (PARTITION BY d.department ORDER BY e.emp_no) AS row_num
# FROM employees e
# JOIN department_summary d ON e.department = d.department
# ORDER BY d.department, e.emp_no
# """ 

# params = None  # You can add parameters if needed

# # Execute the function
# result_df = run_query_with_dynamic_columns(db_configs, user_query, chunk_size=10000, params=params)
# print(result_df)


Error executing query to fetch column names: column "department" does not exist
LINE 3:     SELECT department, COUNT(emp_no) AS num_employees
                   ^
Connection to database Employees closed.
Error processing database Employees: column "department" does not exist
LINE 3:     SELECT department, COUNT(emp_no) AS num_employees
                   ^
No data was retrieved from the databases.
Empty DataFrame
Columns: []
Index: []


In [16]:
import pandas as pd
import psycopg
from contextlib import contextmanager


@contextmanager
def get_connection(config):
    """
    Context manager to handle PostgreSQL connection lifecycle.
    Ensures that the connection is closed after use.
    """
    conn = None
    try:
        conn = psycopg.connect(
            dbname=config["dbname"],
            user=config["user"],
            password=config["password"],
            host=config["host"],
            port=config["port"],
        )
        yield conn
        print(f"Connected to database {config['dbname']} successfully.")
    except psycopg.OperationalError as e:
        print(f"Error connecting to database {config['dbname']}: {e}")
        raise
    finally:
        if conn:
            conn.close()
            print(f"Connection to database {config['dbname']} closed.")
        else:
            print(f"Failed to connect to database {config['dbname']}.")


def fetch_data_in_chunks(conn, query, chunk_size=10000, params=None):
    """
    Fetch data in chunks to handle large datasets.

    :param conn: A psycopg connection object.
    :param query: SQL query to execute.
    :param chunk_size: Number of rows to fetch per chunk.
    :param params: Optional parameters for the SQL query.
    :return: A generator that yields chunks of data.
    """
    with conn.cursor(name="data_cursor") as cursor:
        cursor.execute(query, params)
        while True:
            data = cursor.fetchmany(chunk_size)
            if not data:
                break
            yield data


def fetch_column_names_for_query(conn, query):
    """
    Fetch column names based on the query, supporting complex queries like CTEs, joins, and window functions.
    
    :param conn: A psycopg connection object.
    :param query: SQL query to analyze.
    :return: A list of column names.
    """
    try:
        # Create a simple query that selects no rows to fetch column names
        base_query = f"EXPLAIN (FORMAT JSON) {query}"
        
        with conn.cursor() as cursor:
            cursor.execute(base_query)
            explain_result = cursor.fetchone()

            # Get the output plan from the EXPLAIN result
            if explain_result:
                # Extract the column names from the plan
                plan = explain_result[0]
                column_names = plan[0].get("Plan", {}).get("Output", [])
                print(f"Column names extracted: {column_names}")  # Debug print statement
                return column_names
            else:
                print("Failed to retrieve column names using EXPLAIN.")
                return []

    except Exception as e:
        print(f"Error executing query to fetch column names: {e}")
        raise


def run_query_with_dynamic_columns(db_configs, user_query, chunk_size=10000, params=None):
    """
    Execute a user query across multiple databases and return the results as a pandas DataFrame.

    :param db_configs: List of dictionaries containing database connection parameters.
    :param user_query: SQL query provided by the user.
    :param chunk_size: Number of rows to fetch per chunk.
    :param params: Optional parameters for the SQL query.
    :return: A pandas DataFrame with the combined results from all databases.
             The DataFrame includes a column for the database name.
    """
    dfs = []

    for config in db_configs:
        try:
            with get_connection(config) as conn:
                # Fetch the correct column names based on the user query
                column_names = fetch_column_names_for_query(conn, user_query)

                # Fetch data in chunks and append to DataFrames list
                for chunk in fetch_data_in_chunks(conn, user_query, chunk_size, params):
                    if chunk:
                        # Debug: Print the number of columns in the current chunk
                        print(f"Number of columns in chunk: {len(chunk[0])}")  # Debug print statement
                        
                        # Validate that the number of columns matches the expected number
                        if len(chunk[0]) != len(column_names):
                            print(f"Warning: Number of columns in result from {config['dbname']} does not match the provided column names.")
                            print(f"Expected {len(column_names)} columns, but got {len(chunk[0])} columns.")  # Debug print statement
                            continue

                        for row in chunk:
                            if len(row) != len(column_names):
                                print(f"Warning: Row length mismatch in {config['dbname']}.")
                                continue

                        # Create DataFrame and append to the list
                        df = pd.DataFrame(chunk, columns=column_names)
                        # Optionally, add a column to identify the source database
                        df['database'] = config['dbname']
                        dfs.append(df)
                    else:
                        print(f"No data returned from {config['dbname']}.")
        except Exception as e:
            print(f"Error processing database {config['dbname']}: {e}")

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

    # Log message if no data was retrieved
    if combined_df.empty:
        print("No data was retrieved from the databases.")
    
    return combined_df



# Example usage
db_configs = [
    {
        "dbname": "Employees",
        "user": "postgres",
        "password": "root",
        "host": "localhost",
        "port": "5432",
    }
]

# Sample advanced query (with CTE and window function)
user_query = """
WITH department_summary AS (
    SELECT department, COUNT(emp_no) AS num_employees
    FROM employees
    GROUP BY department
)
SELECT e.emp_no, e.name, d.department, d.num_employees,
       ROW_NUMBER() OVER (PARTITION BY d.department ORDER BY e.emp_no) AS row_num
FROM employees e
JOIN department_summary d ON e.department = d.department
ORDER BY d.department, e.emp_no
"""

params = None  # You can add parameters if needed

# Execute the function
result_df = run_query_with_dynamic_columns(
    db_configs, user_query, chunk_size=10000, params=params
)
print(result_df)


Error executing query to fetch column names: column "department" does not exist
LINE 3:     SELECT department, COUNT(emp_no) AS num_employees
                   ^
Connection to database Employees closed.
Error processing database Employees: column "department" does not exist
LINE 3:     SELECT department, COUNT(emp_no) AS num_employees
                   ^
No data was retrieved from the databases.
Empty DataFrame
Columns: []
Index: []


In [17]:
# Define the user's query
user_query = """SELECT column_name
        FROM information_schema.columns
    WHERE table_name = 'employees';
"""

# Run the function with the user query
df = run_query_with_dynamic_columns(db_configs, user_query)

# Display the combined DataFrame
print(df)




Column names extracted: []
Number of columns in chunk: 1
Expected 0 columns, but got 1 columns.
Connected to database Employees successfully.
Connection to database Employees closed.
No data was retrieved from the databases.
Empty DataFrame
Columns: []
Index: []
